In [1]:
import os, datetime, sys
import dash
import pandas as pd
import numpy as np
import plotly
import matplotlib.pyplot as plt
import cv2
from pdf2image import convert_from_path
import numpy as np
import random
import torch
from sklearn.cluster import KMeans
sys.path.append("..")
from skimage.transform import hough_line, hough_line_peaks

In [2]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

In [3]:
import sklearn as sklearn

In [4]:
from tqdm import tqdm

In [5]:
from tqdm import notebook

In [6]:
import re
from IPython.display import clear_output

rooms = pd.read_excel(os.path.join('DATA/PRETDM-240903-TAB-Planon Export HS 04 Spaces+Equipements.xlsx'))

In [7]:
from functions.params import ROOT,PLAN_ROOT,PLAN_PNG

In [7]:
#Original Hough Lines methods and clustering of the Harris corner based on the angle of there closest line
def detect_harris_corners(image, block_size=2, ksize=3, k=0.04, threshold=0.01, num_clusters=(31*5 +2*4 + 4*3 + 2*2 + 2*1)+(6*6+4*5+3*4+2*4+2*7+3*8+3*9+3*10+2*11+3*12+2*13+1*14+1*15+1*16+1*19)+5, edge_margin=8):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Calculate gradients using Sobel filter
    scale = 1
    delta = 0
    ddepth = cv2.CV_64F
    Ix = cv2.Sobel(gray, ddepth, 1, 0, ksize=5, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
    Iy = cv2.Sobel(gray, ddepth, 0, 1, ksize=5, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
    
    # Calculate products of gradients
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy
    
    # Apply a Gaussian filter to the products of gradients
    sigma = 1
    Ixx = cv2.GaussianBlur(Ixx, (5, 5), sigma)
    Iyy = cv2.GaussianBlur(Iyy, (5, 5), sigma)
    Ixy = cv2.GaussianBlur(Ixy, (5, 5), sigma)
    
    # Compute Harris corner response
    det = (Ixx * Iyy) - (Ixy ** 2)
    trace = Ixx + Iyy
    R = det - k * (trace ** 2)
    
    # Threshold for corner detection
    corners = np.zeros_like(R)
    corners[R > threshold * R.max()] = 255
    
    # Find contours to create a mask for the shape
    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)
    
    # Apply edge margin to the mask
    mask = cv2.erode(mask, np.ones((edge_margin, edge_margin), np.uint8))
    
    # Apply mask to corners
    corners = corners * (mask / 255)
    
    # Get coordinates of the detected corners
    y, x = np.where(corners == 255)
    coordinates = np.column_stack((x, y))
    
    # Apply k-means clustering
    kmeans = KMeans(n_clusters=num_clusters,random_state=7).fit(coordinates)
    centroids = kmeans.cluster_centers_
    
    return centroids

def classify_lines(lines):
    angles = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        # Calculate the angle of the line
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        angles.append(angle)
    return np.array(angles)

def classify_corners_by_lines(corners, lines, angles):
    corners1 = []
    corners2 = []
    
    for corner in corners:
        x, y = corner.ravel()  # Flatten the corner coordinates if necessary
        distances = []
        for line in lines:
            x1, y1, x2, y2 = line[0]
            # Calculate distance from corner to line segment
            distance = np.abs((y2 - y1) * x - (x2 - x1) * y + x2 * y1 - y2 * x1) / np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
            distances.append(distance)
        
        closest_line = np.argmin(distances)
        predominant_angle = angles[closest_line]
        
        if 20 >= predominant_angle >0 or -70 >= predominant_angle:  # Assuming one group has angles < 90 and the other >= 90
            corners1.append(corner)
        else:
            corners2.append(corner)
    
    return np.array(corners1), np.array(corners2)

In [ ]:
#Original Hough Lines methods and clustering of the Harris corner based on the angle of there closest line
image = cv2.imread('Grid_test4.png')
image2 = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

edges = cv2.Canny(gray, 50, 150, apertureSize=3)

img = cv2.imread('Grid_test4.png')
image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
mask = np.zeros_like(img)
points = np.array([[146, 244], [338, 144], [1427, 708],[1561,973]])
cv2.fillPoly(mask, np.int32([points]), color=(0, 255, 0))
mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
coordinates = np.column_stack(np.where(mask_gray > 0))
coordinates = coordinates[:, [1, 0]].astype(int)

for threshold in tqdm(range(125,126,5)):
    for minLineLength in tqdm(range(100,106,5)):
        clear_output(wait=True)
        for maxLineGap in tqdm(range(5,20,5)):
            for n_neighbors in range(1,21,4):
                for rho in [1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2]:
                    for theta in range(40,110,5):
                        threshold = int(threshold)
                        minLineLength = int(minLineLength)
                        maxLineGap = int(maxLineGap)
                        rho=float(rho)
                        theta=int(theta)
                        HoughLines = cv2.HoughLinesP(edges, rho=rho, theta=(np.pi /theta), threshold=threshold,minLineLength=minLineLength,maxLineGap=maxLineGap)
                        horizontal_lines = []
                        vertical_lines = []
                        helper = 0
                        if HoughLines is not None:
                            for i in HoughLines:
                                x1,y1,x2,y2= i[0]
                                if np.degrees(np.arctan2(y2 - y1, x2 - x1))>0:
                                    horizontal_lines.append(i)
                                else:
                                    vertical_lines.append(i)
                        else:
                            print('HoughLines not detected')
                            pass
                        
                        # Draw the Hough lines on the image
                        hough_image = image.copy()
                        if HoughLines is not None:
                            for line in HoughLines:
                                x1,y1,x2,y2= line[0]
                                if np.degrees(np.arctan2(y2 - y1, x2 - x1))>0:
                                    cv2.line(hough_image, (x1,y1),(x2,y2),(0,255,0),2)
                                else:
                                    cv2.line(hough_image, (x1,y1),(x2,y2),(0,255,255),2)
                            
                            # Classify lines based on their angles
                            angles = classify_lines(HoughLines)
                            
                            # Detect Harris corners
                            corners = detect_harris_corners(image)
                            
                            # Classify corners based on the predominant line angles within clusters
                            corners1, corners2 = classify_corners_by_lines(corners, HoughLines, angles)
                
                
                        try:
                            expanded_coordinates = np.expand_dims(coordinates, axis=1)
                            expanded_corners1 = np.expand_dims(corners1.astype(int), axis=0)
                            matches = np.all(expanded_coordinates == expanded_corners1, axis=-1)
                            matching_indices = np.any(matches, axis=1)
                            false_clusters = coordinates[matching_indices]
                
                            expanded_coordinates = np.expand_dims(coordinates, axis=1)
                            expanded_corners2 = np.expand_dims(corners2.astype(int), axis=0)
                            matches = np.all(expanded_coordinates == expanded_corners2, axis=-1)
                            matching_indices = np.any(matches, axis=1)
                            correct_clusters = coordinates[matching_indices]
                            if len(corners1) > 0:
                                if len(corners2) > 0:
                                    if (((len(correct_clusters)/len(corners2) * 100) >= 98.3) and (len(false_clusters) < 18) and (len(corners2)>173)):
                                        helper = helper+1
                                        plt.figure(figsize=(30, 15))
                                        plt.subplot(2, 2, 1)
                                        plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
                                        plt.title(f'Hough Lines - threshold {threshold} - minLineLength {minLineLength} - maxLineGap {maxLineGap} - Total CLusters2 Detected : {len(corners2)} - Clusters in the Plan 2 : {len(correct_clusters)+len(false_clusters)}. {round(len(correct_clusters) / len(corners2) * 100,2)}% of correct -Clusters 2- in Plan 2 Corners. Number of Cluster1 in Parameter 2 : {len(false_clusters)}')
                                        if len(corners1) > 0:
                                            plt.scatter(corners1[:, 0], corners1[:, 1], c='red', marker='o', label='Plan 1 Corners')
                                        else:
                                            print('Only one type of cluster detected')
                                            pass
                                        if len(corners2) > 0:
                                            plt.scatter(corners2[:, 0], corners2[:, 1], c='blue', marker='x', label='Plan 2 Corners')
                                        else:
                                            print('Only one type of cluster detected')
                                            pass
                                        handles, labels = plt.gca().get_legend_handles_labels()
                                        if handles:
                                            plt.legend()
                                    else:
                                        plt.figure(figsize=(30, 15))
                                        plt.subplot(2, 2, 1)
                                        plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
                                        plt.title(f'Hough Lines - threshold {threshold} - minLineLength {minLineLength} - maxLineGap {maxLineGap} ')
                                        if len(corners1) > 0:
                                            plt.scatter(corners1[:, 0], corners1[:, 1], c='red', marker='o', label='Plan 1 Corners')
                                        if len(corners2) > 0:
                                            plt.scatter(corners2[:, 0], corners2[:, 1], c='blue', marker='x', label='Plan 2 Corners')
                                        handles, labels = plt.gca().get_legend_handles_labels()
                                        if handles:
                                            plt.legend()
                                        
                                    if len(corners1) > 0 and len(corners2) > 0:
                                        X_train = np.vstack((corners1, corners2))
                                        y_train = np.hstack((np.zeros(len(corners1)), np.ones(len(corners2))))
                                        n_neighbors = int(n_neighbors)
                                        knn = KNeighborsClassifier(n_neighbors=n_neighbors)#9 - 18 - 19 are the best nneighbours generally
                                        knn.fit(X_train, y_train)
                                        y_pred = knn.predict(corners)
                                        corners1 = corners[y_pred == 0]
                                        corners2 = corners[y_pred == 1]
                                    else:
                                        print('Only one type of cluster detected')
                                        pass
                                    # Display the original image with Hough lines
                                    plt.subplot(2, 2, 2)
                                    plt.imshow(cv2.cvtColor(hough_image, cv2.COLOR_BGR2RGB))
                                    plt.title(f'Hough Lines rho={rho}, theta=np.pi/{theta}')
                                    
                                    # Display the corners on the original image
                                    plt.subplot(2, 2, 3)
                                    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                                    plt.scatter(corners1[:, 0], corners1[:, 1], c='red', marker='o', label='Plan 1 Corners')
                                    plt.scatter(corners2[:, 0], corners2[:, 1], c='blue', marker='x', label='Plan 2 Corners')
                                    handles, labels = plt.gca().get_legend_handles_labels()
                                    if handles:
                                        plt.legend()
                                    plt.title(f'Clustering KNN Neighbours {n_neighbors} - Harris Corners from Intersecting Plans')
                                    plt.subplot(2, 2, 4)
                
                                    try:
                                        expanded_coordinates = np.expand_dims(coordinates, axis=1)
                                        expanded_corners1 = np.expand_dims(corners1.astype(int), axis=0)
                                        matches = np.all(expanded_coordinates == expanded_corners1, axis=-1)
                                        matching_indices = np.any(matches, axis=1)
                                        false_clusters = coordinates[matching_indices]
                        
                                        expanded_coordinates = np.expand_dims(coordinates, axis=1)
                                        expanded_corners2 = np.expand_dims(corners2.astype(int), axis=0)
                                        matches = np.all(expanded_coordinates == expanded_corners2, axis=-1)
                                        matching_indices = np.any(matches, axis=1)
                                        correct_clusters = coordinates[matching_indices]
                                        if len(corners1) > 0:
                                            if len(corners2) > 0:
                                                if (((len(correct_clusters)/len(corners2) * 100) >= 98.3) and (len(false_clusters) < 18) and (len(corners2)>173)):
                                                    # Create a copy of the image to draw the polygon
                                                    image_with_polygon = image_rgb.copy()
                                                    cv2.fillPoly(image_with_polygon, np.int32([points]), color=(0, 255, 0))
                                                    plt.imshow(image_with_polygon)
                                                    if len(corners1) > 0:
                                                        plt.scatter(corners1[:, 0], corners1[:, 1], c='red', marker='o', label='Plan 1 Corners')
                                                    if len(corners2) > 0:
                                                        plt.scatter(corners2[:, 0], corners2[:, 1], c='blue', marker='x', label='Plan 2 Corners')
                                                    handles, labels = plt.gca().get_legend_handles_labels()
                                                    if handles:
                                                        plt.legend()
                                                    plt.title(f'Total CLusters2 Detected : {len(corners2)} - Clusters in the Plan 2 : {len(correct_clusters)+len(false_clusters)}. {round(len(correct_clusters) / len(corners2) * 100,2)}% of correct -Clusters 2- in Plan 2 Corners. Number of Cluster1 in Parameter 2 : {len(false_clusters)}')
                                                    plt.savefig(os.path.join(os.getcwd(),'Presentation','Algorithm_Prediction_Improvement_Improvement',f"Post_analysis_hyperparameters_Horizontal_&_vertical_Hough_Lines_{rho}_{theta}_Clustered_Harris_Corners {threshold} - {minLineLength} - {maxLineGap}, KNN_Clustering neighbours{n_neighbors}.png"))
                                                    plt.close()
                                                    pass
                                                elif helper == 1:
                                                    if os.path.exists((os.path.join(os.getcwd(),'Presentation','Algorithm_Prediction_Improvement_Improvement',f"Attention_Post_analysis_hyperparameters_Horizontal_&_vertical_Hough_Lines_{rho}_{theta}_Clustered_Harris_Corners {threshold} - {minLineLength} - {maxLineGap}.png"))):
                                                        print('File already exists')
                                                        plt.close()
                                                        pass
                                                    else:
                                                        plt.savefig(os.path.join(os.getcwd(),'Presentation','Algorithm_Prediction_Improvement_Improvement',f"Attention_Post_analysis_hyperparameters_Horizontal_&_vertical_Hough_Lines_{rho}_{theta}_Clustered_Harris_Corners {threshold} - {minLineLength} - {maxLineGap}.png"))
                                                        plt.close()
                                                    pass
                                                else:
                                                    print(f'Total CLusters2 Detected : {len(corners2)} - Clusters in the Plan 2 : {len(correct_clusters)+len(false_clusters)}. {round(len(correct_clusters) / len(corners2) * 100,2)}% of correct -Clusters 2- in Plan 2 Corners. Number of -Cluster1- in Parameter 2 : {len(false_clusters)}. Hough_Lines_{rho}_{round(np.pi /theta,4)}_Clustered_Harris_Corners {threshold} - {minLineLength} - {maxLineGap}, KNN_Clustering neighbours{n_neighbors}')
                                                    plt.close()
                                            else:
                                                print(f'len(corners2) not > 0')
                                                plt.close()
                                        else:
                                            print(f'len(corners1) not > 0')
                                            plt.close()
                                    except ValueError as error:
                                        print(error.args)
                                        plt.close()
                                        pass
                        except ValueError as error:
                            print(error.args)
                            plt.close()
                            pass